In [10]:
import pandas as pd
import numpy as np
import bz2

In [30]:
#creating a function to load the text and labels from train and test set

In [12]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9])-1)
        texts.append(x[10:].strip())
    return np.array(labels),texts
train_labels, train_texts = get_labels_and_texts('train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('test.ft.txt.bz2')

In [13]:
train_labels[0]

1

In [14]:
train_texts[0]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [15]:
train_labels = train_labels[0:500]

In [16]:
train_texts = train_texts[0:500]

In [39]:
#text pre-processing

In [17]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalize_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation  = NON_ALPHANUM.sub(r'',lower)
        no_non_ascii = NON_ASCII.sub(r'',lower)
        normalize_texts.append(no_non_ascii)
    return normalize_texts

train_texts = normalize_texts(train_texts)
test_texts  = normalize_texts(test_texts)

In [18]:
train_texts[0]

'stuning even for the nongamer this sound track was beautiful it paints the senery in your mind so well i would recomend it even to people who hate vid game music i have played the game chrono cross but out of all of the games i have ever played it has the best music it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras it would impress anyone who cares to listen '

In [19]:
#countvectorizer

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(train_texts)
X = cv.transform(train_texts)
X_test = cv.transform(test_texts)

In [24]:
X_test

<400000x6217 sparse matrix of type '<class 'numpy.int64'>'
	with 18277251 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, train_labels,train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print("Accuracy for C=%s: %s"
         %(c,accuracy_score(y_val,lr.predict(X_val))))

Accuracy for C=0.01: 0.744
Accuracy for C=0.05: 0.752
Accuracy for C=0.25: 0.776
Accuracy for C=0.5: 0.776
Accuracy for C=1: 0.776


In [29]:
lr.predict(X_test[29])

array([0])

In [31]:
test_labels[29]

0

In [33]:
test_texts[29]

'three days of use and it broke very disappointed in this product it worked perfectly for exactly three days and could not be resuscitated it was very inexpensive so i did not want to pay half again the price to ship it back for an exchange so the company would do nothing when they sent me an inquiry as to product satisfaction'

In [39]:
test_texts[90]

'painful this book has to be one of the most tedious works of literature ever written hawthorne is a great writer but i dont know how this book made it into that sacred list we call classics perhaps on the merit of his name alone'

In [41]:
lr.predict(X_test[90])

array([1])